In [224]:
import pandas as pd 

In [256]:
df_merged = pd.read_csv("df_merged.csv")
#df_merged.head()
#print(df_merged.dtypes)

In [47]:
#df_merged[""].value_counts()                       
#df_merged["video"]


## Retrait de toutes les variables dont on a pas besoin pour l'analyse 

In [257]:
# Variables à supprimer
variables_a_supprimer = ['titre_normalise', 'nom_du_realisateur_bis', 'adult','backdrop_path',
                        'homepage',"original_title","poster_path","status","title","video","nom_du_realisateur_y","tagline",
                        "imdb_id", "Box office"]

# Supprimer les variables inutiles
df_modified = df_merged.drop(variables_a_supprimer, axis=1)


In [238]:
#print(df_modified.dtypes)

In [51]:
#regarder le nombre de valeurs manquantes par variable 
masque_valeurs_manquantes = df_modified.isnull()
valeurs_manquantes_par_colonne = masque_valeurs_manquantes.sum()
print("Nombre de valeurs manquantes par variable :")
print(valeurs_manquantes_par_colonne)

Nombre de valeurs manquantes par variable :
Titre original                        0
Note press                         2076
Notes spectateur                      0
id allocine                           0
Box office                         1925
date                                260
duree_minutes                        39
Nombre_de_critiques_presse         2076
Nombre_de_notes_spectateurs           0
Nombre_de_critiques_spectateurs       0
mois_sortie_x                       260
annee_sortie_x                      260
nom_du_realisateur_x                  3
belongs_to_collection              2963
budget                                0
genres                                5
id                                    0
imdb_id                               1
original_language                     0
overview                              3
popularity                            0
production_companies                 80
production_countries                 28
release_date                        

## Nettoyage des variables 

### Variable genre 

In [ ]:
df_modified["genres"].unique()

# Supposons que df_modified soit votre dataframe et 'genres' la colonne contenant les genres
# Divisez les genres en colonnes de dummies
df_dummies = df_modified['genres'].str.get_dummies(', ')

# Groupby sur l'index et sommez les valeurs (évite les duplications)
df_dummies_grouped = df_dummies.groupby(df_dummies.index).sum()

# Ajoutez les colonnes de dummies à votre dataframe d'origine
df_modified = pd.concat([df_modified, df_dummies_grouped], axis=1)

# Supprimez la colonne 'genres' si nécessaire
#df_modified = df_modified.drop('genres', axis=1)

# Affichez le résultat
# Affichez le DataFrame avec les colonnes encodées
df_modified.head()
df_modified.to_csv("df_modified.csv", index=False)

### Variable durée

In [259]:
# pour visualiser un peu plus clairement
colonnes_a_garder=["Titre original", "id allocine", "id","duree_minutes", "runtime"]
df_duree = df_modified.loc[:, colonnes_a_garder].copy()
df_duree["difference"]=abs(df_duree["duree_minutes"]-df_duree["runtime"])
df_duree.to_csv("df_duree.csv", index=False)
#df_duree["difference"].value_counts()

In [260]:
#je remplace deja les valeurs manquantes de chaque variable par la valeur de l'autre colonne 

# Remplacer les valeurs manquantes dans duree_minutes par les valeurs de runtime
df_duree['duree_minutes'].fillna(df_duree['runtime'], inplace=True)
# Remplacer les valeurs manquantes dans runtime par les valeurs de duree_minutes
df_duree['runtime'].fillna(df_duree['duree_minutes'], inplace=True)

In [244]:
# on remarque que certains films ont plusieurs observations, on va créer une dataframe 
#qui regroupe seulement ces valeurs dupliques pour visualiser un peu plus 

# Créer une nouvelle DataFrame avec les doublons seulement
df_doublons = df_duree[df_duree.duplicated(subset=["id allocine"],keep=False)]
df_doublons.to_csv("df_doublons.csv", index=False)


In [271]:
# apres avoir visualise on voit qu'on peut garder uniquement l'occurence ou la difference 
#de duree est minimale

nom_colonne_titre = "Titre original"
nom_colonne_duree = "difference"
df_duree = df_duree.sort_values(by=[nom_colonne_titre, nom_colonne_duree], ascending=[True, True])

# Affichez le DataFrame trié
#df_duree.head()

df_duree = df_duree.drop_duplicates(subset="id allocine", keep="first")
len(df_duree)

3530

In [272]:
#on voit que la base des differences est beaucoup plus réduite et plus abordable pour 
#faire le tri
df_duree["difference"].value_counts()

difference
0.0      1780
1.0       812
4.0       197
2.0       193
3.0       144
5.0       117
6.0        50
7.0        40
8.0        37
10.0       23
9.0        18
11.0       13
12.0       11
14.0       10
13.0        7
15.0        5
20.0        4
16.0        4
18.0        3
21.0        3
24.0        2
19.0        2
17.0        2
84.0        1
42.0        1
105.0       1
62.0        1
57.0        1
86.0        1
36.0        1
73.0        1
23.0        1
90.0        1
47.0        1
93.0        1
27.0        1
75.0        1
34.0        1
Name: count, dtype: int64

## JE ME SUIS ARRETE A LA REPRENDRE D'ICI

In [72]:
# Supposons que "difference" est le nom de la colonne
nom_colonne = "difference"
valeur_cible = 66

# Utilisez loc pour sélectionner la ligne où la valeur dans la colonne "difference" est égale à 102
ligne_cible = df_modified.loc[df_modified[nom_colonne] == valeur_cible].index[0]

# Affichez le résultat
print("Ligne où la valeur dans la colonne 'difference' est égale à 102 :")
print(ligne_cible)


Ligne où la valeur dans la colonne 'difference' est égale à 102 :
3538


In [74]:
df_modified["duree_minutes"][3538]
#df_modified["runtime"][3538]

11.0

In [77]:
df_sorted["runtime"][3537]

78.0

In [98]:
# Supposons que "difference" est le nom de la colonne
nom_colonne = "difference"
valeur_cible = 84

# Utilisez loc pour sélectionner la ligne où la valeur dans la colonne "difference" est égale à 102
ligne_cible = df_sans_doublons.loc[df_sans_doublons[nom_colonne] == valeur_cible].index[0]

# Affichez le résultat
print("Ligne où la valeur dans la colonne 'difference' est égale à 102 :")
print(ligne_cible)


Ligne où la valeur dans la colonne 'difference' est égale à 102 :
267


In [99]:

print(df_modified["duree_minutes"][ligne_cible])
print(df_modified["runtime"][ligne_cible])
print(df_modified["Titre original"][ligne_cible])

84.0
0.0
Armstrong


In [101]:
df_filtre=df_sans_doublons.loc[df_sans_doublons["runtime"] !=0]
len(df_filtre)

3525

In [107]:
df_filtre["difference"].value_counts()

difference
0.0     1780
1.0      812
4.0      197
2.0      193
3.0      144
5.0      117
6.0       50
7.0       40
8.0       37
10.0      23
9.0       18
11.0      13
12.0      11
14.0      10
13.0       7
15.0       5
16.0       4
20.0       4
21.0       3
18.0       3
24.0       2
19.0       2
17.0       2
42.0       1
62.0       1
57.0       1
86.0       1
36.0       1
73.0       1
23.0       1
47.0       1
27.0       1
34.0       1
Name: count, dtype: int64

In [141]:
# Supposons que "difference" est le nom de la colonne
nom_colonne = "difference"
valeur_cible = 62

# Utilisez loc pour sélectionner la ligne où la valeur dans la colonne "difference" est égale à 102
ligne_cible = df_filtre.loc[df_filtre[nom_colonne] == valeur_cible].index[0]

# Affichez le résultat
#print("Ligne où la valeur dans la colonne 'difference' est égale à 102 :")
#print(ligne_cible)

print(df_filtre["duree_minutes"][ligne_cible])
print(df_filtre["runtime"][ligne_cible])
print(df_filtre["Titre original"][ligne_cible])

120.0
58.0
Diary of a Wimpy Kid


In [131]:

print(df_filtre["duree_minutes"][ligne_cible])
print(df_filtre["runtime"][ligne_cible])
print(df_filtre["Titre original"][ligne_cible])

139.0
105.0
The Scorpion King 4 - Quest for Power


## AUTRE CHOSE 

In [169]:
#budget
df_modified["budget"].eq(0).sum()

1001

In [205]:
#original language 

In [216]:
df_modified["vote_average"].max()

8.421

In [217]:
#box office 
df_modified["Box office"].isnull().sum()

1925